In [572]:
# imports
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset
from datasets import Features
from datasets import Sequence
from datasets import ClassLabel
from datasets import Value
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertConfig, AutoModelForSequenceClassification, \
    DataCollatorForTokenClassification, TrainingArguments, Trainer
import torch

In [573]:
RES_PATH = os.path.abspath("../resources/data/") + "/"

In [574]:
# def load_dataset_dict(dataset_name, dataset_path, limit=None):
#     class_label = Sequence(ClassLabel(num_classes=3, names=["B", "I", "O"]))
#     id_type = Value(dtype='utf8', id=None)
#
#     with open(dataset_path + dataset_name + "/train.json", "r", encoding="utf-8") as file_train:
#         raw_train = pd.read_json(file_train, encoding="utf-8", orient="index").head(limit).fillna("").rename(columns={"label": "labels"})
#     with open(dataset_path + dataset_name + "/test.json", "r", encoding="utf-8") as file_test:
#         raw_test = pd.read_json(file_test, encoding="utf-8", orient="index").head(limit).fillna("").rename(columns={"label": "labels"})
#     with open(dataset_path + dataset_name + "/dev.json", "r", encoding="utf-8") as file_dev:
#         raw_dev = pd.read_json(file_dev, encoding="utf-8", orient="index").head(limit).fillna("").rename(columns={"label": "labels"})
#
#     return datasets.DatasetDict({
#         "train": Dataset.from_pandas(raw_train, preserve_index=True).rename_column("__index_level_0__", "id").cast_column("labels", class_label).cast_column("id", id_type),
#         "test": Dataset.from_pandas(raw_test, preserve_index=True).rename_column("__index_level_0__", "id").cast_column("labels", class_label).cast_column("id", id_type),
#         "dev": Dataset.from_pandas(raw_dev, preserve_index=True).rename_column("__index_level_0__", "id").cast_column( "labels", class_label).cast_column("id", id_type)
#     })

In [575]:
def load_dataset_dict(dataset_name, dataset_path, limit=None):
    train_file = dataset_path + dataset_name + "/train.json"
    test_file = dataset_path + dataset_name + "/test.json"
    dev_file = dataset_path + dataset_name + "/dev.json"

    return load_dataset("json", data_files={'train':train_file, 'validation':dev_file, 'test':test_file},
                        features=Features({
                            "id": Value(dtype='string', id=None),
                            "label": Sequence(ClassLabel(num_classes=3, names=["B", "I", "O"]), length=-1, id=None),
                            "sentence": Sequence(Value(dtype='string', id=None), length=-1, id=None),
                        })).rename_column("label", "labels")

In [576]:
laptop_dataset_dict = load_dataset_dict("test", RES_PATH)
#rest_dataset_dict = load_dataset_dict("rest", RES_PATH)

Found cached dataset json (C:/Users/jannik/.cache/huggingface/datasets/json/default-52fdde12eb95d486/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/3 [00:00<?, ?it/s]

In [577]:
laptop_dataset_dict["train"][0]

{'id': '0',
 'labels': [0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2],
 'sentence': ['Keyboard',
  'is',
  'great',
  'but',
  'primary',
  'and',
  'secondary',
  'control',
  'buttons',
  'could',
  'be',
  'more',
  'durable',
  '.']}

In [578]:
#label_list = laptop_dataset_dict["train"].features["label"].feature.names # ["B", "I", "O"]
label_list = ["B", "I", "O"]
label_list

['B', 'I', 'O']

In [579]:
bert_auto_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [580]:
example = laptop_dataset_dict["train"][0]
tokenized_input = bert_auto_tokenizer(example["sentence"], is_split_into_words=True,padding='max_length', truncation=True, max_length=87)
tokens = bert_auto_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'keyboard',
 'is',
 'great',
 'but',
 'primary',
 'and',
 'secondary',
 'control',
 'buttons',
 'could',
 'be',
 'more',
 'durable',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [581]:
def tokenize_and_align_labels(batch_data):
    tokenized_inputs = bert_auto_tokenizer(batch_data["sentence"], is_split_into_words=True, padding='max_length', truncation=True, max_length=True)

    labels = []
    for i, label in enumerate(batch_data["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100 -> ignored by PT
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [582]:
#tokenized_rest_dataset_dict = rest_dataset_dict.map(tokenize_and_align_labels, batched=True)
tokenized_laptop_dataset_dict = laptop_dataset_dict.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [583]:
tokenized_laptop_dataset_dict["train"].features

{'id': Value(dtype='string', id=None),
 'labels': Sequence(feature=ClassLabel(names=['B', 'I', 'O'], id=None), length=-1, id=None),
 'sentence': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [584]:
data_collator = DataCollatorForTokenClassification(tokenizer=bert_auto_tokenizer, padding=True, max_length=True)

In [585]:
#clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
#clf_metrics = evaluate.load("seqeval")
#clf_metrics = evaluate.evaluator("text-classification")
#ls = [label_list[i] for i in example["labels"]]
#clf_metrics.compute(predictions=[ls], references=[ls])
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [586]:
labels = [label_list[i] for i in example["labels"]]

In [587]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_predictions.append(label_list[p])

    true_labels = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_labels.append(label_list[l])

    true_labels = [label2id[item] for item in true_labels]
    true_predictions = [label2id[item] for item in true_predictions]

    results = {}
    results.update(accuracy_metric.compute(predictions=true_predictions, references=true_labels))
    results.update(precision_metric.compute(predictions=true_predictions, references=true_labels, average='micro'))
    results.update(recall_metric.compute(predictions=true_predictions, references=true_labels, average='micro'))
    results.update(f1_metric.compute(predictions=true_predictions, references=true_labels, average='micro'))

    return results

In [588]:
id2label = {
    0: "B",
    1: "I",
    2: "O",
}
label2id = {
    "B": 0,
    "I": 1,
    "O": 2
}

In [589]:
bert_token_classificator = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

In [590]:
training_args = TrainingArguments(
    output_dir="distilbert-base-uncased_auto-seq-model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    #save_steps=100.0,
    #save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False)

In [591]:
trainer = Trainer(
    model=bert_token_classificator,
    args=training_args,
    train_dataset=tokenized_laptop_dataset_dict["train"],
    eval_dataset=tokenized_laptop_dataset_dict["test"],
    tokenizer=bert_auto_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics)

In [592]:
trainer.train()

C:\Users\jannik\programming_projects\nlp-project-ae\venv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
C:\Users\jannik\programming_projects\nlp-project-ae\venv\lib\site-packages\transformers\tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=2, training_loss=1.1292027235031128, metrics={'train_runtime': 2.9992, 'train_samples_per_second': 10.003, 'train_steps_per_second': 0.667, 'total_flos': 258759060300.0, 'train_loss': 1.1292027235031128, 'epoch': 1.0})

In [593]:
#ps=trainer.predict(laptop_dataset_dict_enc["dev"])
#print(ps.predictions.shape, ps.label_ids.shape)
#bert_auto_tokenizer.convert_tokens_to_ids(tokens)

In [594]:
#trainer.train()

In [595]:
trainer.evaluate()

C:\Users\jannik\programming_projects\nlp-project-ae\venv\lib\site-packages\transformers\tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 1.0521329641342163,
 'eval_accuracy': 0.48507462686567165,
 'eval_precision': 0.48507462686567165,
 'eval_recall': 0.48507462686567165,
 'eval_f1': 0.48507462686567165,
 'eval_runtime': 0.2582,
 'eval_samples_per_second': 30.98,
 'eval_steps_per_second': 3.872,
 'epoch': 1.0}

In [596]:
#trainer.evaluate(laptop_dataset_dict["dev"])

In [597]:
# model evaluation with metrics precision, recall, accuracy, f1-score and confusion matrix
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html